In [1]:
# importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

In [2]:
#1
#read data set

meta_data = pd.read_csv('/Users/zhiyi/Desktop/607/metadata.csv')
print('Original Size of Data:',meta_data.shape)

#drop rows with null values (based on abstract attribute)
meta_data.dropna(subset = ['abstract'],axis = 0, inplace = True)
print('Data Size after dropping rows with null values (based on abstract attribute):',meta_data.shape)

/Users/zhiyi/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Original Size of Data: (1056660, 19)
Data Size after dropping rows with null values (based on abstract attribute): (821118, 19)


In [3]:
#handling duplicate data (based on 'sha','title' and 'abstract')
print(meta_data[meta_data.duplicated(subset=['sha','title','abstract'], keep=False) == True])
meta_data.drop_duplicates(subset=['sha','title','abstract'],keep ='last',inplace=True)
print('Data Size after dropping duplicated data (based on abstract attribute):',meta_data.shape)

         cord_uid  sha                source_x  \
7528     6r98lq0t  NaN                     PMC   
7529     8qcd85x7  NaN                     PMC   
10554    j0mb9zr4  NaN                     PMC   
10710    zhw8vh3e  NaN                     PMC   
10805    smm5s0ai  NaN                     PMC   
...           ...  ...                     ...   
1055884  lob7rary  NaN            Medline; PMC   
1056362  ejprabi5  NaN            Medline; PMC   
1056463  h37h7tgm  NaN  Elsevier; Medline; PMC   
1056511  g5vqg0k8  NaN            Medline; PMC   
1056586  65doyfvd  NaN            Medline; PMC   

                                                     title  \
7528     Management in Ausnahmesituationen: Taktisches ...   
7529     Management in Ausnahmesituationen: Taktisches ...   
10554    Infektionsschutzrecht nach Inkrafttreten des M...   
10710    Artificial Intelligence (AI) applications for ...   
10805    Infektionsschutzrecht nach Inkrafttreten des M...   
...                        

In [4]:
#3
#function to deal with null values
#'No Information Available' will be replaced
def dealing_with_null_values(dataset):
    dataset = dataset
    for i in dataset.columns:
        replace = []
        data  = dataset[i].isnull()
        count = 0
        for j,k in zip(data,dataset[i]):
            if (j==True):
                count = count+1
                replace.append('No Information Available')
            else:
                replace.append(k)
        print("Num of null values (",i,"):",count)
        dataset[i] = replace
    return dataset

meta_data = dealing_with_null_values(meta_data)

Num of null values ( cord_uid ): 0
Num of null values ( sha ): 453723
Num of null values ( source_x ): 0
Num of null values ( title ): 101
Num of null values ( doi ): 247393
Num of null values ( pmcid ): 468365
Num of null values ( pubmed_id ): 376968
Num of null values ( license ): 0
Num of null values ( abstract ): 0
Num of null values ( publish_time ): 1723
Num of null values ( authors ): 4697
Num of null values ( journal ): 76604
Num of null values ( mag_id ): 780393
Num of null values ( who_covidence_id ): 460430
Num of null values ( arxiv_id ): 766222
Num of null values ( pdf_json_files ): 453723
Num of null values ( pmc_json_files ): 509070
Num of null values ( url ): 221696
Num of null values ( s2_id ): 55458


In [5]:
meta_data.shape

(780393, 19)

In [6]:
from sklearn.decomposition import PCA
def pca_fun(n_components, data):
    pca = PCA(n_components=n_components).fit(data)
    data = pca.transform(data)
    return data

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(data):
    tfidf = TfidfVectorizer( stop_words='english',use_idf=True)
    tfidf_matrix = tfidf.fit_transform(data)
    return tfidf_matrix

In [8]:
# Let's create a matrix with tfidf for the column abstract
tfidf_matrix = tfidf(meta_data['abstract'])

In [9]:
# in order to explore which documents have more similar respresentaiton, consine simliartiy can be used
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

# 10 most related documents indices
related_docs_indices = cosine_similarities.argsort()[:-11:-1]
print("Related Document:",related_docs_indices)

# Cosine Similarties of related documents
print("Cosine Similarites of related documents",cosine_similarities[related_docs_indices])

Related Document: [     0 501058 108413 318824   4862 522730 581986 471695 128297 625805]
Cosine Similarites of related documents [1.         0.38452337 0.35538984 0.35166117 0.34838092 0.34520779
 0.34285459 0.34049942 0.33661918 0.33469852]


In [10]:
# Let's take a look at two most similar document
meta_data.iloc[0]['abstract']

'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were 

In [16]:
!pip install wordcloud
!pip install --upgrade pip
!pip install --upgrade Pillow

from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(meta_data.iloc[0]['abstract'])
plt.imshow(wordcloud, interpolation="bilinear")

     |████████████████████████████████| 2.1 MB 3.2 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


ValueError: Only supported for TrueType fonts

In [17]:
meta_data.iloc[17863]['abstract']

'Respiratory syncytial virus (RSV) is one of the main pathogens associated with lower respiratory tract infections in infants and young children worldwide. Exosomes secreted by antigen presenting cells (APCs) can elicit immune responses by carrying major histocompatibility complex (MHC) class I molecules complexed with antigenic peptides and other co-stimulating factors. Therefore, we developed novel immunomagnetic nanographene particles to sequentially isolate, surface engineer, and release intact dendritic cell (DC) exosomes for use as a potential vaccine platform against RSV. The H-2D(b)-restricted, immunodominant peptides from RSV (M(187–195) and NS1(61–75)) were introduced to MHC-I on DC-derived exosomes to express peptide/MHC-I (pMHC-I) complexes. A mouse model of RSV infection was used to define the immunogenicity of surface engineered exosomes for activating virus-specific immune responses. Ex vivo assays demonstrated that engineered exosomes carrying RSV-specific peptides can 

In [18]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(meta_data.iloc[17863]['abstract'])
plt.imshow(wordcloud, interpolation="bilinear")

ValueError: Only supported for TrueType fonts

In [19]:
import gensim
from gensim.models import Doc2Vec

def doc2vec():
    document_tagged = []
    tagged_count = 0
    for _ in meta_data['abstract'].values:
        document_tagged.append(gensim.models.doc2vec.TaggedDocument(_,[tagged_count]))
        tagged_count +=1 
    d2v = Doc2Vec(document_tagged)
    d2v.train(document_tagged,epochs=d2v.epochs,total_examples=d2v.corpus_count)
    return d2v.docvecs.vectors_docs

In [ ]:
# Let's explore the data representation of doc2vec for abstract column.
# Visualization the doc2vec representation
%time doc2vec = doc2vec()

In [ ]:
import seaborn as sns
plt.figure(figsize=(16,16))
sns.heatmap(doc2vec,cmap="coolwarm")